In [4]:
import pandas as pd
import polars as pl
import numpy as np
import mfgdbconnect_py as mfgc
import plotly.express as px
import math
import datetime as dt
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Data Pull (Model)

In [5]:
data_mps = pl.read_excel(
    source = "../output/output_20231024214820.xlsx", 
    sheet_name = "MPS")
data_mps = data_mps.fill_null(0)
data_mps = data_mps.with_columns((pl.col("production_actual") - pl.col("actual_production_non_AL")).alias("actual_production_AL"))
# data_mps = data_mps.rename({"production_plan":"production_plan_AL"})
data_target_rate = pl.read_excel(
    source = "../input/F23_AL_CHEWY.xlsx", 
    sheet_name= "line_rate")
data_product_mapping = pl.read_excel(
    source = "../input/F23_AL_CHEWY.xlsx", 
    sheet_name= "product_mapping")
df_product_mapping = pl.read_excel(
    source = "../input/F23_AL_CHEWY.xlsx",
    sheet_name = "product_mapping"
)

In [6]:
df_prod_plan = pl.read_excel(source = "../output/output_20231024214820.xlsx", sheet_name = "production_plan")
df_prod_plan = df_prod_plan.join(df_product_mapping,on="material_cd",how = "left")
df_prod_plan = df_prod_plan.drop(columns = "material_cd")
df_prod_plan = df_prod_plan.melt(
    id_vars=["product_name"], 
    value_vars = [str(i) for i in range(335)] , 
    variable_name = "day_number",
    value_name = "production_status"
).with_columns(pl.col("day_number").cast(pl.Int16))
df_prod_plan = df_prod_plan.with_columns((pl.col("day_number")/7).floor().cast(pl.Int64).alias("week_nbr"))
df_prod_plan = df_prod_plan.join(
    data_mps[["product_name","week_nbr","production_plan_AL"]], 
    on = ["product_name","week_nbr"], 
    how = "left"
)
df_prod_weekly = df_prod_plan.groupby(["product_name","week_nbr"]).agg([
    pl.sum("production_status").alias("total_days")
]).sort(by = "week_nbr")
df_prod_plan = df_prod_plan.join(df_prod_weekly,on = ["product_name","week_nbr"])
df_prod_plan  = df_prod_plan.with_columns((pl.col("production_status")/pl.col("total_days")).alias("contribution"))
df_prod_plan = df_prod_plan.fill_nan(0)
df_prod_plan = df_prod_plan.with_columns((pl.col("contribution")*pl.col("production_plan_AL")).alias("production_plan"))
df_prod_plan.head()

product_name,day_number,production_status,week_nbr,production_plan_AL,total_days,contribution,production_plan
str,i16,i64,i64,i64,i64,f64,f64
"""CH TRAIL MIX F…",0,0,0,0,0,0.0,0.0
"""CH REESES TREA…",0,0,0,0,0,0.0,0.0
"""CH SNS ALMOND …",0,1973857,0,3059044,3059044,0.645253,1.9739e6
"""CH SNS PEANUT …",0,0,0,0,0,0.0,0.0
"""CH SNS DK CHOC…",0,0,0,0,0,0.0,0.0


In [7]:
df_run_length_lists = []
df_cycle_list = []
for product_name in df_prod_plan["product_name"].unique():
    df = df_prod_plan.filter(pl.col("product_name") == product_name).sort(by = "day_number")
    df = df.with_columns(pl.when(pl.col("production_plan") > 0).then(1).otherwise(0).alias("true_prouction_status"))
    df = df.with_columns(
        pl.col("true_prouction_status")
        .shift(1)
        .alias("production_status_prev_day")).fill_null(0)
    df = df.with_columns(
        pl.when((pl.col("true_prouction_status") == 1) & (pl.col("production_status_prev_day") == 0))
        .then(1)
        .otherwise(0)
        .alias("production_start_status"))
    df = df.with_columns(pl.col("production_start_status").cumsum().alias("cycle_id"))
    df = df.with_columns(
        pl.when(pl.col("true_prouction_status")  == 1)
        .then(pl.col("cycle_id"))
        .otherwise(None)
        .alias("production_id"))
    df = df.with_columns(
        pl.when(pl.col("true_prouction_status") == 1)
        .then(None)
        .otherwise(pl.col("cycle_id"))
        .alias("cycle_id")
    )
    df = df.with_columns(pl.lit(1).alias("run_length"))
    df = df.with_columns(pl.lit(1).alias("cycle_days"))
    df_run = df.groupby(["product_name","production_id"]).agg([pl.count("run_length")]).drop_nulls()
    df_cycle = df.groupby(["product_name","cycle_id"]).agg([pl.count("cycle_days")]).drop_nulls()
    df_run_length_lists.append(df_run)
    df_cycle_list.append(df_cycle)
df_run_length = pl.concat(df_run_length_lists, how="vertical_relaxed")
df_cycle_time = pl.concat(df_cycle_list, how="vertical_relaxed")
display(df_run_length.head())
display(df_cycle_time.head())

product_name,production_id,run_length
str,i32,u32
"""CH PROTEIN PB …",1,1
"""CH PROTEIN PB …",2,1
"""CH PROTEIN PB …",7,1
"""CH PROTEIN PB …",8,1
"""CH PROTEIN PB …",3,1


product_name,cycle_id,cycle_days
str,i32,u32
"""CH PROTEIN PB …",0,14
"""CH PROTEIN PB …",1,8
"""CH PROTEIN PB …",4,1
"""CH PROTEIN PB …",7,1
"""CH PROTEIN PB …",10,2


In [8]:
df_run_length_summary = df_run_length.groupby(["product_name"]).agg(
    [
        pl.mean("run_length").round(1).alias("mean_run_length_model"),
        pl.std("run_length").round(1).alias("std_run_length_model"),
        pl.max("run_length").alias("max_run_length_model"),
        pl.min("run_length").alias("min_run_length_model")
    ]
)
df_cycle_time_summary = df_cycle_time.groupby(["product_name"]).agg(
    [
        pl.mean("cycle_days").round(1).alias("mean_cycle_days_model"),
        pl.std("cycle_days").round(1).alias("std_cycle_days_model"),
        pl.max("cycle_days").alias("max_cycle_days_model"),
        pl.min("cycle_days").alias("min_cycle_days_model")
    ]
)
df_run_length_cycle_time_model = df_run_length_summary.join(df_cycle_time_summary,on = "product_name")
df_run_length_cycle_time_model.head()

product_name,mean_run_length_model,std_run_length_model,max_run_length_model,min_run_length_model,mean_cycle_days_model,std_cycle_days_model,max_cycle_days_model,min_cycle_days_model
str,f64,f64,u32,u32,f64,f64,u32,u32
"""CH SNS CHOCOLA…",1.3,0.5,2,1,40.8,61.7,164,1
"""CH SNS ALMOND …",2.4,2.1,8,1,25.3,25.4,67,1
"""CH TRAIL MIX F…",4.3,3.1,9,1,44.1,60.5,176,1
"""CH SNS PEANUT …",11.5,13.2,42,2,20.0,23.1,77,1
"""CH SNS DK CHOC…",1.9,1.1,4,1,31.8,70.2,228,1


#### Actual Production

In [9]:
data_act_production = pd.read_excel("../input/F23_AL_CHEWY.xlsx", sheet_name = "actual_production_afo")
data_act_production["Date"] = pd.to_datetime(data_act_production["Date"])
data_act_production = pl.DataFrame(data_act_production)
data_act_production = data_act_production.filter(pl.col("Plant") == "AL")
data_act_production = data_act_production.select(["Parent Product Name","Date","Actual Production EA"]).rename({
    "Parent Product Name":"product_name",
    "Date":"date",
    "Actual Production EA":"actual_production_ea"
})
date_list = []
product_list = []
for date in pl.date_range(dt.datetime(2022,5,30), dt.datetime(2023,5,28), "1d", eager=True,time_unit="ns"):
    for product in data_act_production["product_name"].unique():
        date_list.append(date)
        product_list.append(product)
df = pl.DataFrame(pd.DataFrame({"date":date_list,"product_name":product_list}))
data_act_production = df.join(data_act_production,on = ["date","product_name"], how="left")
data_act_production = data_act_production.groupby(["product_name","date"]).agg([pl.sum("actual_production_ea")])
data_act_production.head()

product_name,date,actual_production_ea
str,datetime[ns],f64
"""CH SNS ALMOND …",2022-05-30 00:00:00,1.500624e6
"""CH SNS DK CHOC…",2022-05-30 00:00:00,0.0
"""CH SNS CHOCOLA…",2022-05-31 00:00:00,0.0
"""CH TRAIL MIX F…",2022-05-31 00:00:00,0.0
"""CH SNS PEANUT …",2022-05-31 00:00:00,0.0


In [10]:
df_run_length_lists = []
df_cycle_list = []
for product_name in data_act_production["product_name"].unique():
    df = data_act_production.filter(pl.col("product_name") == product_name).sort(by = "date")
    df = df.with_columns(pl.when(pl.col("actual_production_ea") > 0).then(1).otherwise(0).alias("true_prouction_status"))
    df = df.with_columns(
        pl.col("true_prouction_status")
        .shift(1)
        .alias("production_status_prev_day")).fill_null(0)
    df = df.with_columns(
        pl.when((pl.col("true_prouction_status") == 1) & (pl.col("production_status_prev_day") == 0))
        .then(1)
        .otherwise(0)
        .alias("production_start_status"))
    df = df.with_columns(pl.col("production_start_status").cumsum().alias("cycle_id"))
    df = df.with_columns(
        pl.when(pl.col("true_prouction_status")  == 1)
        .then(pl.col("cycle_id"))
        .otherwise(None)
        .alias("production_id"))
    df = df.with_columns(
        pl.when(pl.col("true_prouction_status") == 1)
        .then(None)
        .otherwise(pl.col("cycle_id"))
        .alias("cycle_id")
    )
    df = df.with_columns(pl.lit(1).alias("run_length"))
    df = df.with_columns(pl.lit(1).alias("cycle_days"))
    df_run = df.groupby(["product_name","production_id"]).agg([pl.count("run_length")]).drop_nulls()
    df_cycle = df.groupby(["product_name","cycle_id"]).agg([pl.count("cycle_days")]).drop_nulls()
    df_run_length_lists.append(df_run)
    df_cycle_list.append(df_cycle)
df_run_length = pl.concat(df_run_length_lists, how="vertical_relaxed")
df_cycle_time = pl.concat(df_cycle_list, how="vertical_relaxed")
display(df_run_length.head())
display(df_cycle_time.head())

product_name,production_id,run_length
str,i32,u32
"""CH PROTEIN PB …",3,1
"""CH PROTEIN PB …",5,1
"""CH PROTEIN PB …",1,3
"""CH PROTEIN PB …",2,8
"""CH PROTEIN PB …",4,5


product_name,cycle_id,cycle_days
str,i32,u32
"""CH PROTEIN PB …",0,20
"""CH PROTEIN PB …",3,45
"""CH PROTEIN PB …",4,12
"""CH PROTEIN PB …",1,1
"""CH PROTEIN PB …",2,28


In [11]:
df_run_length_summary = df_run_length.groupby(["product_name"]).agg(
    [
        pl.mean("run_length").round(1).alias("mean_run_length_actual"),
        pl.std("run_length").round(1).alias("std_run_length_actual"),
        pl.max("run_length").alias("max_run_length_actual"),
        pl.min("run_length").alias("min_run_length_actual")
    ]
)
df_cycle_time_summary = df_cycle_time.groupby(["product_name"]).agg(
    [
        pl.mean("cycle_days").round(1).alias("mean_cycle_days_actual"),
        pl.std("cycle_days").round(1).alias("std_cycle_days_actual"),
        pl.max("cycle_days").alias("max_cycle_days_actual"),
        pl.min("cycle_days").alias("min_cycle_days_actual")
    ]
)
df_run_length_cycle_time_act = df_run_length_summary.join(df_cycle_time_summary,on = "product_name")
df_run_length_cycle_time_act.head()

product_name,mean_run_length_actual,std_run_length_actual,max_run_length_actual,min_run_length_actual,mean_cycle_days_actual,std_cycle_days_actual,max_cycle_days_actual,min_cycle_days_actual
str,f64,f64,u32,u32,f64,f64,u32,u32
"""CH SNS CDA DK …",2.8,2.1,6,1,49.6,83.1,234,3
"""CH SNS DK CHOC…",5.2,5.2,15,1,47.6,85.9,238,1
"""CH PROTEIN PB …",3.6,3.0,8,1,57.7,90.6,240,1
"""CH SNS ALMOND …",3.1,2.7,8,1,22.9,42.3,164,1
"""CH SNS PEANUT …",6.1,6.6,19,1,15.7,18.2,70,1


In [12]:
df_run_length_cycle_time = df_run_length_cycle_time_act.join(
    df_run_length_cycle_time_model,
    on = "product_name",
    how = "outer"
)
df_run_length_cycle_time.head()

product_name,mean_run_length_actual,std_run_length_actual,max_run_length_actual,min_run_length_actual,mean_cycle_days_actual,std_cycle_days_actual,max_cycle_days_actual,min_cycle_days_actual,mean_run_length_model,std_run_length_model,max_run_length_model,min_run_length_model,mean_cycle_days_model,std_cycle_days_model,max_cycle_days_model,min_cycle_days_model
str,f64,f64,u32,u32,f64,f64,u32,u32,f64,f64,u32,u32,f64,f64,u32,u32
"""CH SNS CHOCOLA…",3.3,0.6,4,3,88.5,51.3,154,34,1.3,0.5,2,1,40.8,61.7,164,1
"""CH SNS ALMOND …",3.1,2.7,8,1,22.9,42.3,164,1,2.4,2.1,8,1,25.3,25.4,67,1
"""CH TRAIL MIX F…",3.4,3.3,13,1,18.2,15.7,53,2,4.3,3.1,9,1,44.1,60.5,176,1
"""CH SNS PEANUT …",6.1,6.6,19,1,15.7,18.2,70,1,11.5,13.2,42,2,20.0,23.1,77,1
"""CH SNS DK CHOC…",5.2,5.2,15,1,47.6,85.9,238,1,1.9,1.1,4,1,31.8,70.2,228,1


In [13]:
# df_run_length_cycle_time.write_excel("mct.xlsx")

#### Utilization

In [14]:
df_target_rate = data_target_rate.join(data_product_mapping, on = "material_cd", how = "left")
df_utilization_plan = df_prod_plan.join(df_target_rate, on = "product_name", how = "left")
df_utilization_plan = df_utilization_plan.with_columns(
    pl.when(pl.col("production_status") == 1)
    .then(pl.col("line_rate_per_day"))
    .otherwise(0)
    .alias("line_rate_per_day")
)
df_utilization_plan = df_utilization_plan.with_columns((pl.col("production_plan")/pl.col("line_rate_per_day")).alias("utilization"))
df_utilization_plan.head()

product_name,day_number,production_status,week_nbr,production_plan_AL,total_days,contribution,production_plan,material_cd,line_rate_per_day,utilization
str,i16,i64,i64,i64,i64,f64,f64,i64,f64,f64
"""CH TRAIL MIX F…",0,0,0,0,0,0.0,0.0,4104398000,0.0,NaN
"""CH REESES TREA…",0,0,0,0,0,0.0,0.0,4125119120,0.0,NaN
"""CH SNS ALMOND …",0,1973857,0,3059044,3059044,0.645253,1.9739e6,4127706000,0.0,inf
"""CH SNS PEANUT …",0,0,0,0,0,0.0,0.0,4127707000,0.0,NaN
"""CH SNS DK CHOC…",0,0,0,0,0,0.0,0.0,4127855000,0.0,NaN


#### Gant Chart of Production Plan & Actual Production

In [15]:
df = df_prod_plan.with_columns(
    pl.when(pl.col("production_plan") > 0).then(pl.col("product_name")).otherwise(None).alias("active_product")
).drop_nulls()
fig = px.scatter(df,x = "day_number", y = "active_product", title = "Production Status by Day (Model)")
fig

In [16]:
df = data_act_production.with_columns(
    pl.when(pl.col("actual_production_ea") > 0).then(pl.col("product_name")).otherwise(None).alias("active_product")
).drop_nulls()
fig = px.scatter(df,x = "date", y = "active_product", title = "Production Status by Day (Actual)")
fig

In [17]:
fig = go.Figure(data=[
    go.Bar(
        name='Run Length in Days (Model)', 
        x = df_run_length_cycle_time["product_name"], 
        y = df_run_length_cycle_time["mean_run_length_model"],
        text = df_run_length_cycle_time["mean_run_length_model"]
    ),
    go.Bar(
        name='Run Length in Days (Actual)', 
        x = df_run_length_cycle_time["product_name"], 
        y = df_run_length_cycle_time["mean_run_length_actual"],
        text = df_run_length_cycle_time["mean_run_length_actual"]
    ),
    go.Bar(
        name='Mean Cycle Time in Days (Model)', 
        x = df_run_length_cycle_time["product_name"], 
        y = df_run_length_cycle_time["mean_cycle_days_model"],
        text = df_run_length_cycle_time["mean_cycle_days_model"]
    ),
    go.Bar(
        name='Mean Cycle Time in Days (Actual)', 
        x = df_run_length_cycle_time["product_name"], 
        y = df_run_length_cycle_time["mean_cycle_days_actual"],
        text = df_run_length_cycle_time["mean_cycle_days_actual"]
    )
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [18]:
df = data_mps.groupby("product_name").agg([pl.mean("demand")]).sort(by = "demand", descending=True)
px.bar(df,x = "product_name",y = "demand")

#### Product Wise Analysis

In [19]:
product_list = list(df_run_length_cycle_time["product_name"].unique())
print(product_list)
product_name = "CH SNS PEANUT PARENT"
product_name

['CH PROTEIN PB DK CHOC PARENT', 'CH TRAIL MIX F&N PARENT', 'CH SNS CHOCOLATE PRETZEL NUT PARENT', 'CH SNS ALMOND PARENT', 'CH SNS DK CHOC PEA ALM PARENT', 'CH SNS CDA DK CHOC PEA  ALM PARENT', 'CH SNS PEANUT PARENT']


'CH SNS PEANUT PARENT'

In [20]:
df = data_mps.filter(pl.col("product_name") == product_name)
agg_cols = ["production_plan_AL","production_actual","actual_production_non_AL",
            "actual_production_AL","demand","closing_inventory_plan",
            "closing_inventory_actual_calculated","safety_stock","inventory_capacity"]
df = df.groupby(["week_nbr"]).agg(
    [
        pl.sum(col) for col in agg_cols
    ]
).sort(by = "week_nbr")
bar_cols = ["production_plan_AL","production_actual","actual_production_non_AL","actual_production_AL"]
line_cols = ["demand","closing_inventory_plan","closing_inventory_actual_calculated","safety_stock", "inventory_capacity"]
fig = go.Figure()
for col in bar_cols:
    fig.add_trace(
        go.Bar(
            name=col, 
            x=df["week_nbr"], 
            y=df[col]
        )
    )
for col in line_cols:
    fig.add_trace(
        go.Line(
            name=col, 
            x=df["week_nbr"], 
            y=df[col]
        )
    )
# Change the bar mode
fig.update_xaxes(title = "Week Number")
fig.update_layout(barmode='group',title = product_name)
fig.show()

c:\Users\G666966\OneDrive - General Mills\.virtualenvs\production_scheduling\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [22]:
# df_utilization_plan["production_plan"].sum()/df_utilization_plan["line_rate_per_day"].sum()

In [24]:
# df = df_utilization_plan.filter(pl.col("product_name") == product_name)
group_by_col = "week_nbr"
df = df_utilization_plan.groupby(group_by_col).agg(
    [
        (pl.sum("production_plan")*100/pl.sum("line_rate_per_day")).round().alias("utilization")
    ]
).sort(by = group_by_col)
px.bar(df, x = group_by_col, y = "utilization", text="utilization", title = "Utilization % (Actual Production/Line Rate) by Week")

#### Model Outputs

In [23]:
# df_prod_plan = pl.read_excel(source = "../Output/output_20230806174322.xlsx", sheet_name = "daily_production_plan")
# df_prod_plan = df_prod_plan.drop(columns = "material_cd")
# df_prod_plan = df_prod_plan.melt(
#     id_vars=["product_name"], 
#     value_vars = [str(i) for i in range(335)] , 
#     variable_name = "day_number",
#     value_name = "production_status"
# ).with_columns(pl.col("day_number").cast(pl.Int16))
# df_prod_plan = df_prod_plan.with_columns((pl.col("day_number")/7).floor().cast(pl.Int64).alias("week_nbr"))
# df_prod_plan = df_prod_plan.join(
#     data_mps[["product_name","week_nbr","production_plan_AL"]], 
#     on = ["product_name","week_nbr"], 
#     how = "left"
# )
# df_prod_weekly = df_prod_plan.group_by(["product_name","week_nbr"]).agg([
#     pl.sum("production_status").alias("total_days")
# ]).sort(by = "week_nbr")
# df_prod_plan = df_prod_plan.join(df_prod_weekly,on = ["product_name","week_nbr"])
# df_prod_plan  = df_prod_plan.with_columns((pl.col("production_status")/pl.col("total_days")).alias("contribution"))
# df_prod_plan = df_prod_plan.fill_nan(0)
# df_prod_plan = df_prod_plan.with_columns((pl.col("contribution")*pl.col("production_plan_AL")).alias("production"))
# df_prod_plan.head()

In [24]:
# df_co = data_co
# df_co = df_co.with_columns(
#     [
#         pl.col("period").cast(pl.Float64),
#         pl.col("CO_Time").cast(pl.Float64)
#     ]
# )
# df_co = df_co.filter(pl.col("CO_Flag") == "TRUE").sort(by = "period")
# df_co = df_co.with_columns(pl.col("period").shift(1).alias("period_start"))
# df_co = df_co.with_columns(pl.col("period").alias("period_end"))
# df_co = df_co.with_columns(pl.col("period").diff().alias("run_length")).fill_null(value = 0)
# df_co.head()

In [25]:
# df_run_length_model = df_co.group_by("from_product_name").agg(
#     [
#         pl.mean("run_length").round(1).alias("mean_run_length_model"),
#         pl.max("run_length").round(1).alias("max_run_length_model"),
#         pl.min("run_length").round(1).alias("min_run_length_model"),
#         pl.std("run_length").round(1).alias("standard_deviation_run_length_model"),
#         pl.count("run_length").alias("count_runs_model")
#     ]
# )
# df_run_length_model

In [26]:
# df_cycle_time_actual = data_prod_log.with_columns(pl.when(pl.col("activity") != "NP").then("CO").otherwise("NP").alias("activity"))
# df_cycle_time_actual = df_cycle_time_actual.with_columns(pl.col("activity").shift(1).alias("prev_activity"))
# df_cycle_time_actual = df_cycle_time_actual.with_columns(
#     pl.when((pl.col("activity") == "CO") & (pl.col("prev_activity") == "NP"))
#     .then(1)
#     .otherwise(0)
#     .alias("co_start_status")
# )
# df_cycle_time_actual = df_cycle_time_actual.with_columns(
#     pl.col("co_start_status").cumsum().alias("co_id")
# )
# df_cycle_time_actual = df_cycle_time_actual.with_columns(
#     pl.when(pl.col("activity") == "NP").then(None).otherwise(pl.col("co_id")).alias("co_id")
# )
# df_cycle_time_actual = df_cycle_time_actual.group_by(["product_name","co_id"]).agg(
#     [
#         pl.col("production_duration").sum().alias("cycle_days")
#     ]
# ).drop_nulls()
# df_cycle_time_actual = df_cycle_time_actual.group_by(["product_name"]).agg(
#     [
#         pl.mean("cycle_days").round(1).alias("mean_cycle_days_act"),
#         pl.max("cycle_days").round(1).alias("max_cycle_days_act"),
#         pl.min("cycle_days").round(1).alias("min_cycle_days_act"),
#         pl.std("cycle_days").round(1).alias("std_cycle_days_act")
#     ]
# )
# df_run_length_actual = data_prod_log.filter(pl.col("activity") == "NP")
# df_run_length_actual = df_run_length_actual.with_columns(
#     (pl.col("production_duration")/24).alias("run_length_days"))
# df_run_length_actual = df_run_length_actual.group_by("product_name").agg(
#     [
#         pl.mean("run_length_days").round(1).alias("mean_run_length_act"),
#         pl.max("run_length_days").round(1).alias("max_run_length_act"),
#         pl.min("run_length_days").round(1).alias("min_run_length_act"),
#         pl.std("run_length_days").round(1).alias("standard_deviation_run_length_act"),
#         pl.count("run_length_days").alias("count_runs_actual")
#     ]
# )
# df_run_length_cycle_time_act = df_run_length_actual.join(
#     df_cycle_time_actual,
#     on = "product_name"
# )
# df_run_length_cycle_time_act